In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('../input/bri-data-hackathon-pa/train.csv')
test = pd.read_csv('../input/bri-data-hackathon-pa/test.csv')

In [3]:
len_df_train=train.shape[0]
data_train_test= pd.concat([train, test], sort=False)

In [4]:
def missing(dff):
    print (round((dff.isnull().sum() * 100/ len(dff)),2).sort_values(ascending=False))

missing(data_train_test)

Best Performance                         34.98
Last_achievement_%                        0.01
Achievement_above_100%_during3quartal     0.01
number_of_dependences                     0.00
job_duration_in_current_job_level         0.00
person_level                              0.00
job_duration_in_current_person_level      0.00
job_duration_in_current_branch            0.00
Employee_type                             0.00
gender                                    0.00
age                                       0.00
marital_status_maried(Y/N)                0.00
Education_level                           0.00
GPA                                       0.00
year_graduated                            0.00
job_duration_from_training                0.00
branch_rotation                           0.00
job_rotation                              0.00
assign_of_otherposition                   0.00
annual leave                              0.00
sick_leaves                               0.00
job_level    

In [5]:
data_train_test.drop(['GPA','Last_achievement_%'],axis=1,inplace=True)
data_train_test.drop(['person_level','Education_level'],axis=1,inplace=True)

In [6]:
for col in ["gender"]:
    data_train_test[col]= data_train_test[col].map({2: 'male',
                                  1:'female'})

In [7]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in data_train_test.columns if
                    data_train_test[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in data_train_test.columns if 
                data_train_test[cname].dtype in ['int64', 'float64']]

In [8]:
for i in numerical_cols:
    data_train_test[i]=data_train_test[i].fillna(data_train_test[i].mean())
for i in categorical_cols:
    data_train_test[i]=data_train_test[i].fillna(data_train_test[i].mode()[0])

In [9]:
# # Get number of unique entries in each column with categorical data
# object_nunique = list(map(lambda col: data_train_test[col].nunique(), categorical_cols))
# d = dict(zip(categorical_cols, object_nunique))

# # Print number of unique entries by column, in ascending order
# sorted(d.items(), key=lambda x: x[1])

In [10]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(data_train_test[categorical_cols]))

# One-hot encoding removed index; put it back
OH_cols.index = data_train_test.index


# Remove categorical columns (will replace with one-hot encoding)
num_X = data_train_test.drop(categorical_cols, axis=1)

# Add one-hot encoded columns to numerical features
data_train_test = pd.concat([num_X, OH_cols], axis=1)

In [11]:
#split the data again
X_train_full= data_train_test[:len_df_train]
X_test_full= data_train_test[len_df_train:]
y_train = X_train_full['Best Performance']
X_train_full.drop(['Best Performance'],axis=1,inplace=True)
X_test_full.drop(['Best Performance'],axis=1,inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
# X_train_full.drop([2435,4383,5042,5691,6905,8799,8967,9285,9428,10126,10617], axis=0)

In [13]:
# X_train_full.drop([10334,6261], axis=0)

In [14]:
import xgboost as xgb
from xgboost import XGBClassifier

In [15]:
## Hyper Parameter Optimization

params={
 "n_estimators"     : np.arange(0,1500),
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 9, 10, 11, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ],
    
}

In [16]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [17]:
# def timer(start_time=None):
#     if not start_time:
#         start_time = datetime.now()
#         return start_time
#     elif start_time:
#         thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
#         tmin, tsec = divmod(temp_sec, 60)
#         print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [18]:
classifier=xgb.XGBClassifier( objective ='binary:logistic',
    eval_metric='auc')

In [19]:
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [20]:
random_search.fit(X_train_full,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  1.2min finished


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           eval_metric='auc', gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None...
                                           subsample=None, tree_method=None,
                                           validate_parameters=None,
                                   

In [21]:
random_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, eval_metric='auc',
              gamma=0.3, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=5, missing=nan,
              monotone_constraints='()', n_estimators=1202, n_jobs=0,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [22]:
# from bayes_opt import BayesianOptimization
# import warnings
# warnings.filterwarnings('ignore')

In [23]:
# optimizer = BayesianOptimization(classifier, params, random_state=1)

In [24]:
model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, eval_metric='auc',
              gamma=0.2, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.15, max_delta_step=0,
              max_depth=4, min_child_weight=1,
              monotone_constraints='()', n_estimators=360, n_jobs=0,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [25]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_train_full, y_train, cv=4, scoring="roc_auc", n_jobs=-1)

print("Average MAE score:", scores.mean())

Average MAE score: 0.539351164117138


In [26]:
X_train_full.drop([10334,6261], axis=0)

,job_duration_in_current_job_level,job_duration_in_current_person_level,job_duration_in_current_branch,age,number_of_dependences,year_graduated,job_duration_from_training,branch_rotation,job_rotation,assign_of_otherposition,...,1,2,3,4,5,6,7,8,9,10
0,1.352775,1.352775,1.732051,1985,1,2011,4,2,3,2,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1.292285,1.292285,1.039230,1989,0,2007,4,4,3,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,2.565151,2.308679,1.780449,1970,1,1989,28,10,9,6,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
3,2.828427,1.000000,1.385641,1967,2,1987,29,11,6,0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,2.828427,2.828427,0.707107,1965,3,1985,30,9,8,2,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11148,1.256981,1.256981,1.224745,1986,1,2016,4,3,3,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
11149,1.385641,1.385641,1.802776,1988,2,2015,3,1,2,0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
11150,1.256981,1.256981,1.385641,1991,0,2017,2,1,2,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
11151,1.118034,1.118034,0.000000,1987,1,2012,5,5,4,0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [27]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_train_full, y_train, cv=5, scoring="roc_auc", n_jobs=-1)

print("Average MAE score:", scores.mean())

Average MAE score: 0.5483366063185929


In [28]:
print(len(X_train_full))

11153


In [29]:
# data_check_x = X_train_full
# data_check_y = y_train

In [30]:
# import time
# start_time = time.time()

# akurasi = []
# iteration = np.arange(0,11153)
# for i in iteration:
#     data_check_x = data_check_x.drop([i], axis=0)
#     data_check_y = data_check_y.drop([i], axis=0)
#     scores = cross_val_score(model, data_check_x, data_check_y,
#                               cv=4)
#     ava_score = scores.mean()
#     akurasi.append(ava_score)
#     data_check_x = X_train_full
#     data_check_y = y_train

# plt.figure(figsize=(20,10))
# plt.scatter(iteration,akurasi)

# print("--- %s seconds ---" % (time.time() - start_time))

In [31]:
# data = {'iteration':  iteration,
#         'akurasi': akurasi}

# df = pd.DataFrame (data, columns = ['iteration','akurasi'])

In [32]:
model.fit(X_train_full, y_train)
preds_test = model.predict_proba(X_test_full)[:,1]

In [33]:
X_test_full['index'] = X_test_full.index
X_test_full.set_index('index')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,job_duration_in_current_job_level,job_duration_in_current_person_level,job_duration_in_current_branch,age,number_of_dependences,year_graduated,job_duration_from_training,branch_rotation,job_rotation,assign_of_otherposition,...,1,2,3,4,5,6,7,8,9,10
index,,,,,,,,,,,,,,,,,,,,,
0,1.256981,1.256981,0.707107,1988,0,2009,6,4,2,6,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.224745,1.224745,1.256981,1991,0,2014,2,2,2,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,0.500000,0.500000,1.081665,1989,1,2011,4,2,2,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,1.256981,1.256981,1.802776,1990,1,2014,3,1,3,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,1.352775,1.352775,1.224745,1985,0,2007,5,3,2,0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,1.292285,1.292285,0.707107,1988,0,2015,2,2,2,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
5996,1.352775,1.352775,1.581139,1989,1,2011,5,3,2,7,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
5997,1.385641,1.385641,1.224745,1991,0,2013,3,2,3,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [34]:
output = pd.DataFrame({'index': X_test_full.index,
                        'Best Performance': preds_test})
output.to_csv('submission.csv', index=False)